In [1]:
pip install pymongo opencv-python face_recognition numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import face_recognition
import pymongo

In [3]:
# Módulo 1: Registro de Usuarios
class UserManager:
    def __init__(self):
        self.client = pymongo.MongoClient("mongodb://localhost:27017/")
        self.db = self.client["facial_recognition"]
        self.collection = self.db["users"]

    def is_username_taken(self, username):
        """Verifica si el nombre de usuario ya está registrado."""
        return self.collection.find_one({'username': username}) is not None

    def register_user(self, username, password):
        """Registra un nuevo usuario."""
        if self.is_username_taken(username):
            print("El nombre de usuario ya está tomado.")
            return False

        user_data = {
            'username': username,
            'password': password,
            'face_data': []
        }
        self.collection.insert_one(user_data)
        print("Registro exitoso.")
        return True

In [4]:
# Módulo 2: Captura de Rostro
class FaceCapture:
    def __init__(self, user_manager):
        self.user_manager = user_manager

    def capture_face(self, username):
        """Captura y almacena la imagen facial del usuario."""
        cap = cv2.VideoCapture(0)
        face_data = []

        while True:
            ret, frame = cap.read()
            if ret:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_data.append(rgb_frame)
                self.update_camera_frame(frame)

                if cv2.waitKey(1) & 0xFF == ord('c'):
                    break

        cap.release()
        cv2.destroyAllWindows()

        if face_data:
            rgb_face_data = [cv2.cvtColor(img, cv2.COLOR_RGB2BGR) for img in face_data]
            self.user_manager.collection.update_one({'username': username}, {'$set': {'face_data': [img.tolist() for img in rgb_face_data]}})
            print("Captura de rostro exitosa.")
        else:
            print("No se capturó ninguna imagen.")

    def update_camera_frame(self, frame):
        # Este método puede ser implementado en la GUI
        pass

In [5]:
# Módulo 3: Verificación de Usuarios
class UserVerification:
    def __init__(self, user_manager):
        self.user_manager = user_manager

    def verify_user(self, username, password):
        """Verifica el usuario y la contraseña."""
        user_data = self.user_manager.collection.find_one({'username': username})
        if user_data and user_data['password'] == password:
            return user_data
        return None

    def verify_face(self, username):
        """Verifica la identificación facial del usuario a partir de la imagen capturada."""
        user_data = self.user_manager.collection.find_one({'username': username})
        stored_face_data = user_data['face_data']

        if not stored_face_data:
            print("No hay datos faciales almacenados.")
            return False

        stored_face_data = np.array(stored_face_data[0], dtype=np.uint8)
        stored_face_encodings = face_recognition.face_encodings(stored_face_data)

        if not stored_face_encodings:
            print("No se pudieron obtener las codificaciones faciales.")
            return False

        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        cap.release()

        if not ret:
            print("No se pudo capturar la imagen.")
            return False

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        captured_face_encodings = face_recognition.face_encodings(rgb_frame)

        if not captured_face_encodings:
            print("No se pudieron obtener las codificaciones faciales capturadas.")
            return False

        match = face_recognition.compare_faces([stored_face_encodings[0]], captured_face_encodings[0])

        if match[0]:
            print("Reconocimiento facial exitoso.")
            return True
        else:
            print("Reconocimiento facial fallido.")
            return False

In [7]:
# Módulo 4: Interfaz de Usuario (GUI)
class FacialRecognitionGUI:
    def __init__(self, master, user_manager, face_capture, user_verification):
        self.master = master
        self.user_manager = user_manager
        self.face_capture = face_capture
        self.user_verification = user_verification
        master.title("Facial Recognition Login")
        master.geometry("400x500")
        master.configure(bg="#f0f0f0")

        # Crear un marco para la cámara
        self.camera_frame = tk.Frame(master, bg="#f0f0f0")
        self.camera_frame.pack(pady=20)

        # Crear un label para mostrar la imagen de la cámara
        self.camera_label = tk.Label(self.camera_frame, bg="#f0f0f0")
        self.camera_label.pack()

        # Crear los campos de entrada y los botones
        self.username_label = tk.Label(master, text="Usuario", bg="#f0f0f0", font=("Arial", 12))
        self.username_label.pack(pady=10)

        self.username_entry = tk.Entry(master, font=("Arial", 12), bg="white", relief=tk.SOLID, borderwidth=1)
        self.username_entry.pack(pady=10)

        self.password_label = tk.Label(master, text="Contraseña", bg="#f0f0f0", font=("Arial", 12))
        self.password_label.pack(pady=10)

        self.password_entry = tk.Entry(master, show="*", font=("Arial", 12), bg="white", relief=tk.SOLID, borderwidth=1)
        self.password_entry.pack(pady=10)

        self.login_button = tk.Button(master, text="Iniciar Sesión", bg="#4CAF50", fg="white", font=("Arial", 12), command=self.login)
        self.login_button.pack(pady=10)

        self.register_button = tk.Button(master, text="Registrarse", bg="#2196F3", fg="white", font=("Arial", 12), command=self.register)
        self.register_button.pack(pady=10)

        self.update_camera_frame()

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        if self.user_verification.verify_user(username, password):
            if self.user_verification.verify_face(username):
                print("Inicio de sesión exitoso.")
            else:
                print("Error en el reconocimiento facial.")
        else:
            print("Error de inicio de sesión.")

    def register(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        if self.user_manager.register_user(username, password):
            self.face_capture.capture_face(username)

    def update_camera_frame(self):
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        cap.release()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            photo = ImageTk.PhotoImage(Image.fromarray(frame))
            self.camera_label.configure(image=photo)
            self.camera_label.image = photo
        self.camera_label.after(10, self.update_camera_frame)

In [8]:
# Módulo Principal
if __name__ == "__main__":
    user_manager = UserManager()
    face_capture = FaceCapture(user_manager)
    user_verification = UserVerification(user_manager)

    root = tk.Tk()
    gui = FacialRecognitionGUI(root, user_manager, face_capture, user_verification)
    root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_23444\1445160134.py", line 45, in login
    if self.user_verification.verify_user(username, password):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_23444\1013512275.py", line 8, in verify_user
    user_data = self.user_manager.collection.find_one({'username': username})
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py", line 1730, in find_one
    for result in cursor.limit(-1):
                  ^^^^^^^^^^^^^^^^
  File "C:\Users\User\anaconda3\Lib\site-packages\pymongo\synchronous\cursor.py", line 1281, in __next__
    return self.next()
           ^^^^^^^^^^^